In [64]:
import pandas as pd
import numpy as np
import warnings

warnings.filterwarnings("ignore")

In [65]:
sampled_pwds_50 = pd.read_csv("tl/e24_random_50_pwmds.csv")
sampled_pwds_50 = list(sampled_pwds_50['powermode'].values)

sampled_pwds_10 = pd.read_csv("tl/e24_random_10_pwmds.csv")
sampled_pwds_10 = list(sampled_pwds_10['powermode'].values)

In [66]:
model_train = 'lstm'
model_infer = 'yolo'

model_train = 'resnet'
model_infer = 'bert'

arr_rate_list = [30, 40, 50, 60, 70, 80, 90, 100, 110, 120]
time_list = [0.5, 0.6, 0.7, 0.8, 0.9, 1.0, 1.1, 1.2, 1.3, 1.4, 1.5, 1.6, 1.7, 1.8, 1.9, 2.0]
power_list = np.arange(10, 51, 1)

time_list = [2, 2.5, 3, 3.5, 4, 4.5, 5, 5.5, 6]
arr_rate_list = [1,2,3,4,5,6,7,8,9,10, 11, 12, 13, 14, 15]
power_list = np.arange(10, 61, 1)

In [67]:
data_train = pd.read_csv(f'{model_train}_train_data_final.csv')
data_infer = pd.read_csv(f'{model_infer}_infer_data_final.csv')

In [68]:
core_vals=[4, 8, 12] #3 possible values
gpu_vals=[114750000, 318750000, 522750000, 726750000, 930750000, 1134750000, 1300500000]
cpu_vals=[422400, 729600, 1036800, 1344000, 1651200, 1958400, 2201600] #in kHz, 7 possible values
mem_vals = [665600000, 2133000000, 3199000000]
bs_vals = [1, 4, 16, 32, 64]

In [69]:
# # Filter data based on values in the list only
data_train = data_train[data_train['cores'].isin(core_vals)]
data_train = data_train[data_train['gpu'].isin(gpu_vals)]
data_train = data_train[data_train['cpu'].isin(cpu_vals)]
data_train = data_train[data_train['mem'].isin(mem_vals)]
data_train.reset_index(drop=True, inplace=True)

In [70]:
def helper(data, arr_rate):
    data['interleaved_power'] = np.maximum(data['observed_power_train'], data['observed_power_infer'])
    data['interleaved_time'] = data['observed_time_train'] + data['observed_time_infer']
    data['powermode'] = data['cores'].astype(str) + "_" + data['cpu'].astype(str) + "_" + data['gpu'].astype(str) + "_" + data['mem'].astype(str)
    data['observed_time_infer_scaled'] = data['observed_time_infer']/1000
    data['observed_time_train_scaled'] = data['observed_time_train']/1000
    data['num_train_batches'] = np.floor(((data['bs'].astype(int))/arr_rate - data['observed_time_infer_scaled'])/data['observed_time_train_scaled']).astype(int)
    data['time_cond1'] = (data['bs'].astype(int))/arr_rate + data['observed_time_infer_scaled']
    data['time_cond2'] = data['observed_time_infer_scaled']*arr_rate

    return data

In [71]:
def get_best_row_optim(data, time_budget, power_budget):

    data = data[data['interleaved_power'] <= power_budget]
    data = data[data['time_cond1'] <= time_budget]
    data = data[data['time_cond2'] <= data['bs']]


    if data.empty:
        return None, None, None, None, None, None
    
    else:     
        data = data.sort_values(by=['num_train_batches'],ascending=False)

        train_batch_vals = data['num_train_batches'].unique()
        max_train_batch_val = max(train_batch_vals)

        data = data[data['num_train_batches'] == max_train_batch_val]
        data.sort_values(by=['observed_time_infer_scaled'], inplace=True)

        data = data.iloc[0]
        powermode = data['powermode']
        bs = data['bs']
        power = data['interleaved_power']
        infer_time = data['observed_time_infer']
        train_time = data['observed_time_train']
        num_train_batches = data['num_train_batches']

        return powermode, bs, power, infer_time, train_time, num_train_batches

In [72]:
def get_best_row_optim_nn(data, time_budget, power_budget):

    data = data[data['predicted_interleaved_power'] <= power_budget]
    data = data[data['predicted_time_cond1'] <= time_budget]
    data = data[data['predicted_time_cond2'] <= data['bs']]


    if data.empty:
        return None, None, None, None, None, None
    
    else:     
        data = data.sort_values(by=['predicted_num_train_batches'],ascending=False)

        train_batch_vals = data['predicted_num_train_batches'].unique()
        max_train_batch_val = max(train_batch_vals)

        data = data[data['predicted_num_train_batches'] == max_train_batch_val]
        data.sort_values(by=['predicted_time_infer_scaled'], inplace=True)

        data = data.iloc[0]
        powermode = data['powermode']
        bs = data['bs']
        power = data['observed_interleaved_power']
        infer_time = data['observed_time_infer']
        train_time = data['observed_time_train']
        num_train_batches = data['observed_num_train_batches']

        return powermode, bs, power, infer_time, train_time, num_train_batches

In [73]:
nn_train_pareto_df = pd.read_csv(f"/home/saisamarth/exp/tl/e24_merged_observed_predicted_{model_train}_NN_sampled_50.csv")
nn_infer_pareto_df = pd.read_csv(f"/home/saisamarth/exp/tl/e24_merged_observed_predicted_{model_train}_NN_infer_sampled_50.csv")
nn_train_pareto_df = nn_train_pareto_df[nn_train_pareto_df['powermode'].isin(sampled_pwds_50)]
nn_infer_pareto_df = nn_infer_pareto_df[nn_infer_pareto_df['powermode'].isin(sampled_pwds_10)]
nn_train_pareto_df.reset_index(inplace=True, drop=True)
nn_infer_pareto_df.reset_index(inplace=True, drop=True)

In [74]:
# merge nn_train_pareto_df and nn_infer_pareto_df based on powermode with suffixes _train and _infer


In [75]:
def helper_nn(data, arr_rate):
    data['observed_interleaved_power'] = np.maximum(data['observed_power_train'], data['observed_power_infer'])
    data['observed_interleaved_time'] = data['observed_time_train'] + data['observed_time_infer']
    data['predicted_interleaved_power'] = np.maximum(data['predicted_power_train'], data['predicted_power_infer'])
    data['predicted_interleaved_time'] = data['predicted_time_train'] + data['predicted_time_infer']

    data['predicted_time_infer_scaled'] = data['predicted_time_infer']/1000
    data['predicted_time_train_scaled'] = data['predicted_time_train']/1000
    data['observed_time_infer_scaled'] = data['observed_time_infer']/1000
    data['observed_time_train_scaled'] = data['observed_time_train']/1000

    data['predicted_num_train_batches'] = np.floor(((data['bs'].astype(int))/arr_rate - data['predicted_time_infer_scaled'])/data['predicted_time_train_scaled']).astype(int)
    data['observed_num_train_batches'] = np.floor(((data['bs'].astype(int))/arr_rate - data['observed_time_infer_scaled'])/data['observed_time_train_scaled']).astype(int)

    data['predicted_time_cond1'] = (data['bs'].astype(int))/arr_rate + data['predicted_time_infer_scaled']
    data['predicted_time_cond2'] = data['predicted_time_infer_scaled']*arr_rate

    data['observed_time_cond1'] = (data['bs'].astype(int))/arr_rate + data['observed_time_infer_scaled']
    data['observed_time_cond2'] = data['observed_time_infer_scaled']*arr_rate

    return data

In [76]:
nn_result_df = pd.DataFrame(columns=['powermode','infer_time','bs','interleaved_power','power_budget','num_train_batches','time_budget', 'arr_rate'])
# time_list = [1]
# power_list = [20]
for arr_rate in arr_rate_list:
    for time_budget in time_list:
        for power_budget in power_list:
            nn_pareto_df = pd.merge(nn_train_pareto_df, nn_infer_pareto_df, on=['powermode'], suffixes=('_train', '_infer'))
            nn_pareto_df = helper_nn(nn_pareto_df, arr_rate)
            pwd, bs, interleaved_power, infer_time, train_time, best_num_train_batches = get_best_row_optim_nn(nn_pareto_df, time_budget, power_budget)
            nn_result_df.loc[len(nn_result_df)] = [pwd, infer_time, bs, interleaved_power,power_budget, best_num_train_batches, time_budget, arr_rate]

In [77]:
random_result_df = pd.DataFrame(columns=['powermode','infer_time','bs','interleaved_power','power_budget','num_train_batches','time_budget','arr_rate'])
# time_list = [1]
# power_list = [20]
for arr_rate in arr_rate_list:
    for time_budget in time_list:
        for power_budget in power_list:
            # merge the data_train and data_infer based on cores, gpu, cpu, mem with suffixes _train and _infer
            data_all = pd.merge(data_train, data_infer, on=['cores', 'gpu', 'cpu', 'mem'], suffixes=('_train', '_infer'))
            data_all = helper(data_all,arr_rate)
            data_all = data_all[data_all['powermode'].isin(sampled_pwds_10)]
            pwd, bs, interleaved_power, infer_time, train_time, best_num_train_batches = get_best_row_optim(data_all, time_budget, power_budget)
            random_result_df.loc[len(random_result_df)] = [pwd, infer_time, bs, interleaved_power,power_budget, best_num_train_batches, time_budget, arr_rate]

In [78]:
random_result_df.dropna(inplace=True)

In [79]:
random_result_df

,powermode,infer_time,bs,interleaved_power,power_budget,num_train_batches,time_budget,arr_rate
6,8_1344000_114750000_665600000,557.418243,1.0,15.3530,16.0,0.0,2.0,1.0
7,8_1344000_114750000_2133000000,553.883423,1.0,16.1660,17.0,0.0,2.0,1.0
8,12_2201600_114750000_2133000000,552.249084,1.0,17.7560,18.0,0.0,2.0,1.0
9,12_2201600_114750000_2133000000,552.249084,1.0,17.7560,19.0,0.0,2.0,1.0
10,8_1344000_522750000_665600000,238.206413,1.0,19.8780,20.0,3.0,2.0,1.0
...,...,...,...,...,...,...,...,...
5962,4_422400_930750000_3199000000,600.276215,8.0,41.9755,56.0,0.0,6.0,13.0
5963,4_422400_930750000_3199000000,600.276215,8.0,41.9755,57.0,0.0,6.0,13.0
5964,4_422400_930750000_3199000000,600.276215,8.0,41.9755,58.0,0.0,6.0,13.0
5965,4_422400_930750000_3199000000,600.276215,8.0,41.9755,59.0,0.0,6.0,13.0


In [80]:
# merge nn_result_df and random_result_df based on power_budget and time_budget with suffixes _nn and _random
result_df = pd.merge(nn_result_df, random_result_df, on=['power_budget', 'time_budget','arr_rate'], suffixes=('_nn', '_random'))

In [81]:
result_df.to_csv(f'interleaved_{model_train}_train_{model_infer}_infer_nn_alt_random_multi_al.csv', index=False)